In [3]:
import pandas as pd
import pymysql


In [ ]:
import pytest
from pytest import approx

In [7]:
admissions=pd.read_csv('https://raw.githubusercontent.com/espadaone/project/main/datasets/ADMISSIONS.csv')
callout = pd.read_csv('https://raw.githubusercontent.com/espadaone/project/main/datasets/CALLOUT.csv')
caregivers = pd.read_csv('https://raw.githubusercontent.com/espadaone/project/main/datasets/CAREGIVERS.csv')
chartevents = pd.read_csv('https://raw.githubusercontent.com/espadaone/project/main/datasets/CHARTEVENTS.csv')
cptevents = pd.read_csv('https://raw.githubusercontent.com/espadaone/project/main/datasets/CPTEVENTS.csv')
datetimeevents = pd.read_csv('https://raw.githubusercontent.com/espadaone/project/main/datasets/DATETIMEEVENTS.csv')
diagnoses_icd = pd.read_csv('https://raw.githubusercontent.com/espadaone/project/main/datasets/DIAGNOSES_ICD.csv',dtype = 'str')
drgcodes = pd.read_csv('https://raw.githubusercontent.com/espadaone/project/main/datasets/DRGCODES.csv')
d_cpt = pd.read_csv('https://raw.githubusercontent.com/espadaone/project/main/datasets/D_CPT.csv')
d_icd_diagnoses = pd.read_csv('https://raw.githubusercontent.com/espadaone/project/main/datasets/D_ICD_DIAGNOSES.csv')
d_icd_procedures = pd.read_csv('https://raw.githubusercontent.com/espadaone/project/main/datasets/D_ICD_PROCEDURES.csv', dtype = 'str')
d_items = pd.read_csv('https://raw.githubusercontent.com/espadaone/project/main/datasets/D_ITEMS.csv')
d_labelitems = pd.read_csv('https://raw.githubusercontent.com/espadaone/project/main/datasets/D_LABITEMS.csv')
icustays = pd.read_csv('https://raw.githubusercontent.com/espadaone/project/main/datasets/ICUSTAYS.csv')
inputevents_mv = pd.read_csv('https://raw.githubusercontent.com/espadaone/project/main/datasets/INPUTEVENTS_CV.csv')
labevents = pd.read_csv('https://raw.githubusercontent.com/espadaone/project/main/datasets/LABEVENTS.csv')
microbiologyevents = pd.read_csv('https://raw.githubusercontent.com/espadaone/project/main/datasets/MICROBIOLOGYEVENTS.csv')
outputevents = pd.read_csv('https://raw.githubusercontent.com/espadaone/project/main/datasets/OUTPUTEVENTS.csv')
patients = pd.read_csv('https://raw.githubusercontent.com/espadaone/project/main/datasets/PATIENTS.csv')
prescriptions = pd.read_csv('https://raw.githubusercontent.com/espadaone/project/main/datasets/PRESCRIPTIONS.csv')
procedureevents_mv = pd.read_csv('https://raw.githubusercontent.com/espadaone/project/main/datasets/PROCEDUREEVENTS_MV.csv')
procedures_icd = pd.read_csv('https://raw.githubusercontent.com/espadaone/project/main/datasets/PROCEDURES_ICD.csv', dtype = 'str')
services = pd.read_csv('https://raw.githubusercontent.com/espadaone/project/main/datasets/SERVICES.csv')
transfers = pd.read_csv('https://raw.githubusercontent.com/espadaone/project/main/datasets/TRANSFERS.csv')

C:\Users\PABLO\AppData\Local\Temp\ipykernel_7632\4286518172.py:4: DtypeWarning: Columns (8,10,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  chartevents = pd.read_csv('https://raw.githubusercontent.com/espadaone/project/main/datasets/CHARTEVENTS.csv')
C:\Users\PABLO\AppData\Local\Temp\ipykernel_7632\4286518172.py:15: DtypeWarning: Columns (17,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  inputevents_mv = pd.read_csv('https://raw.githubusercontent.com/espadaone/project/main/datasets/INPUTEVENTS_CV.csv')


In [8]:
chartevents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 758355 entries, 0 to 758354
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   row_id        758355 non-null  int64  
 1   subject_id    758355 non-null  int64  
 2   hadm_id       758355 non-null  int64  
 3   icustay_id    758274 non-null  float64
 4   itemid        758355 non-null  int64  
 5   charttime     758355 non-null  object 
 6   storetime     758355 non-null  object 
 7   cgid          758355 non-null  int64  
 8   value         755630 non-null  object 
 9   valuenum      323884 non-null  float64
 10  valueuom      239855 non-null  object 
 11  warning       382279 non-null  float64
 12  error         382279 non-null  float64
 13  resultstatus  21674 non-null   object 
 14  stopped       374649 non-null  object 
dtypes: float64(4), int64(5), object(6)
memory usage: 86.8+ MB


In [3]:
# Renombramos la columna que sera la clave primaria
chartevents.rename(columns={"row_id":"chartevents_id"}, inplace=True)
# Eliminamos la columna con el id del paciente
chartevents.drop(columns="subject_id", inplace=True)
# Eliminamos duplicados
chartevents.drop_duplicates(inplace = True)
#Fechas a tipo datetime
chartevents["charttime"] = pd.to_datetime(chartevents["charttime"])
chartevents["storetime"] = pd.to_datetime(chartevents["storetime"])

#Normalizacion del valor ml por mL
unidad = chartevents["valueuom"] == "ml"
chartevents.loc[unidad,["valueuom"]] = "mL"
#Remplazar los valores nulos por 0 para pasar el tipo de dato a entero, aplique o no el 0 representara lo mismo
chartevents["warning"].fillna(0, inplace=True)
chartevents["warning"] = chartevents["warning"].astype("int64")
chartevents["error"].fillna(0, inplace=True)
chartevents["error"] = chartevents["error"].astype("int64")
#Tambien se agrego un 0 en los campos de icustay_id, esto representa que no hay data sobre el icustay
chartevents["icustay_id"].fillna(0, inplace=True)
chartevents["icustay_id"] = chartevents["icustay_id"].astype("int64")

#Datos rellenados
chartevents["value"].fillna("N/D", inplace=True)
chartevents["resultstatus"].fillna("N/D", inplace=True)
chartevents["stopped"].fillna("N/D", inplace=True)
chartevents["valuenum"].fillna(0, inplace=True)

# Reordena las columnas para que coincidan con las tablas de sql
chartevents = chartevents[ ['chartevents_id', 'hadm_id', 'icustay_id', 'itemid', 'charttime',
                            'storetime', 'cgid', 'value', 'valuenum', 'valueuom', 'warning',
                            'error', 'resultstatus', 'stopped'] ]

In [4]:
chartevents["icustay_id"].fillna(0, inplace=True)

In [5]:
chartevents["valueuom"].fillna('N/D', inplace=True)

In [23]:
chartevents 

,chartevents_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,valueuom,warning,error,resultstatus,stopped
0,5279021,126179,279554,223761,2130-02-04 04:00:00,2130-02-04 04:35:00,19085,95.9,95.9,?F,0,0,N/D,N/D
1,5279022,126179,279554,224695,2130-02-04 04:25:00,2130-02-04 05:55:00,18999,2222221.7,2222221.7,cmH2O,0,0,N/D,N/D
2,5279023,126179,279554,220210,2130-02-04 04:30:00,2130-02-04 04:43:00,21452,15.0,15.0,insp/min,0,0,N/D,N/D
3,5279024,126179,279554,220045,2130-02-04 04:32:00,2130-02-04 04:43:00,21452,94.0,94.0,bpm,0,0,N/D,N/D
4,5279025,126179,279554,220179,2130-02-04 04:32:00,2130-02-04 04:43:00,21452,163.0,163.0,mmHg,0,0,N/D,N/D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758350,274493917,103379,217992,226228,2170-12-19 19:33:00,2170-12-19 19:33:00,20889,F,2.0,N/D,0,0,N/D,N/D
758351,274493918,103379,217992,226381,2170-12-19 19:33:00,2170-12-19 19:33:00,20889,Single,1.0,N/D,0,0,N/D,N/D
758352,274493919,103379,217992,226543,2170-12-19 19:33:00,2170-12-19 19:33:00,20889,Not specified,12.0,N/D,0,0,N/D,N/D
758353,274493920,103379,217992,226544,2170-12-19 19:33:00,2170-12-19 19:33:00,20889,English,1.0,N/D,0,0,N/D,N/D


In [24]:
chart = chartevents.values.tolist()

In [25]:
connection = pymysql.connect(
    host='servaz.mysql.database.azure.com',
    #port=3306,
    user='Azadmin',
    password='AZcosmospf08',
    db='pf_uci'
    #ssl_ca='C:\ETL\conexion_ssl\DigiCertGlobalRootCA.crt.pem'
)

In [26]:

#connection = pymysql.connect(
#    host='servaz.mysql.database.azure.com',
 #   user='Azadmin@se',
#    password='AZcosmospf08',
 #   db='pf_uci',
 #   ssl='C:\ETL\conexion_ssl\DigiCertGlobalRootCA.crt.pem' 
#)
 
#if connection.is_connected():


cursor = connection.cursor()
cursor.executemany("""INSERT INTO chartevents VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)""",chart)

   
connection.commit()
      
#connection.rollback()
#connection.close()
        